In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for Training

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# Define the Class

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


# Model Layers

In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3009,  0.0086,  0.1127,  0.4688, -0.0959, -0.1043,  0.4505,  0.2544,
         -0.2083,  0.3243, -0.5553, -0.1969, -0.7053, -0.2156,  0.4449,  0.0561,
         -0.1545,  0.2885, -0.6607,  0.3888],
        [-0.0033,  0.0350,  0.0760,  0.3630, -0.0178, -0.0238,  0.4613,  0.2113,
         -0.0213, -0.1628, -0.4584, -0.4208, -0.3913,  0.0992,  0.2705,  0.1325,
         -0.0999,  0.2474, -0.5089,  0.4425],
        [ 0.1671, -0.0310, -0.1954,  0.4566, -0.3884, -0.1160, -0.1456,  0.1157,
         -0.3491,  0.1840, -0.6424, -0.3810, -0.5095, -0.1667,  0.4019,  0.0233,
         -0.2413,  0.3123, -0.5963,  0.2831]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3009, 0.0086, 0.1127, 0.4688, 0.0000, 0.0000, 0.4505, 0.2544, 0.0000,
         0.3243, 0.0000, 0.0000, 0.0000, 0.0000, 0.4449, 0.0561, 0.0000, 0.2885,
         0.0000, 0.3888],
        [0.0000, 0.0350, 0.0760, 0.3630, 0.0000, 0.0000, 0.4613, 0.2113, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0992, 0.27

In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
     nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[-0.1114, -0.1453,  0.0957,  0.1203,  0.0374, -0.0425, -0.0158, -0.1851,
         -0.0660,  0.0771],
        [-0.1545, -0.2087,  0.1245,  0.2947, -0.0583, -0.0415, -0.1425, -0.1805,
          0.0234,  0.1470],
        [ 0.0099, -0.0482,  0.0937,  0.1783,  0.0196, -0.0613, -0.1130,  0.0247,
         -0.0222,  0.1801]], grad_fn=<AddmmBackward0>)


In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0911, 0.0881, 0.1121, 0.1149, 0.1058, 0.0976, 0.1003, 0.0847, 0.0954,
         0.1100],
        [0.0863, 0.0817, 0.1141, 0.1352, 0.0950, 0.0966, 0.0873, 0.0841, 0.1031,
         0.1166],
        [0.0980, 0.0924, 0.1065, 0.1159, 0.0989, 0.0912, 0.0866, 0.0994, 0.0949,
         0.1161]], grad_fn=<SoftmaxBackward0>)


# Model Parameters

In [17]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 1.5211e-02,  7.2309e-05,  2.7034e-02,  ..., -8.1781e-03,
         -2.2732e-02, -3.0738e-02],
        [ 2.4532e-02,  7.1489e-03, -7.4539e-03,  ...,  7.8258e-03,
          3.0587e-02,  1.1749e-02]], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0026, -0.0188], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0360, -0.0035,  0.0424,  ..., -0.0053,  0.0050, -0.0403],
        [-0.0043,  0.0296,  0.0370,  ..., -0.01